- Started on July 31, 2019
- Works with a subset of the highd dataset
- Goal is to run particle filtering on this highd dataset with the
ultimate aim of learning idm rules for driver modeling
- Trivia: Timestep is 0.04 sec, each vehicle is visible for median duration of
13.6 seconds. This translates to about 340 timesteps of visibility. We have verified
this by starting from scene 1 and going till 340 looking at truck vehicle id 27

## Needs experimentation
- Should the particle fitness be based on likelihood of hallucinated x location
under Gaussian centered around true x location? Or should this also include
y? Reason for asking this question is because Frenet s is 0 all the time so
we will need to work with posG i.e. global position

## Needs investigation
- Why is Frenet s always 0?

# Import packages, load data, include helpers

In [59]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # Make video in notebook
using Reel # Save video as gif
using CSV # For writing to csv
using DataFrames # For writing to csv
using PyPlot # For in notebook plotting
using Distributions
using Test
using StatsBase # For the weights function in likelihood sampling

In [5]:
# The trajectories i.e. positions of the cars with timestep
traj_highd = open(io->read(io, MIME"text/plain"(), Trajdata), 
    "trajdata_Data11_Road1_nLIOD3_lower.txt", "r");
# The highd roadway geometry
roadway_highd = open(io->read(io, MIME"text/plain"(), Roadway), 
    "highd_roadway.txt", "r");

In [136]:
include("admin_functions.jl"); # Brings in the functions defined in admin_functions.jl

# Workspace

In [217]:
# Let's plot the velocities over time for some vehicle
vels = zeros(1,100)
for i in 1:100
    scene = Scene(500)
    get!(scene,traj_highd,i)
    vels[i] = scene.entities[2].state.v
end
@show vels

vels = [33.0504 33.0604 33.0704 33.0804 33.0903 33.1003 33.1203 33.1303 33.1403 33.1503 33.1602 33.1802 33.1902 33.2002 33.2102 33.2302 33.2402 33.2502 33.2701 33.2801 33.2901 33.3101 33.3201 33.3401 33.3501 33.3601 33.3801 33.3901 33.4101 33.4201 33.4301 33.4501 33.4601 33.4701 33.4901 33.5001 33.5201 33.5301 33.5401 33.5601 33.5701 33.5801 33.5901 33.6101 33.6201 33.6301 32.9402 32.9603 32.9703 32.9803 32.9903 33.0103 33.0203 33.0304 33.0404 33.0504 33.0704 33.0804 33.0905 33.1005 33.1105 33.1305 33.1405 33.1505 33.1606 33.1706 33.1906 33.2006 33.2106 33.2206 33.2407 33.2507 33.2607 33.2707 33.2907 33.3007 33.3106 33.3206 33.3406 33.3506 33.3606 33.3706 33.3905 33.4005 33.4105 33.4305 33.4405 33.4505 33.4705 33.4804 33.4904 28.16 28.16 28.17 28.18 28.18 28.19 28.2 28.2 28.21]


1×100 Array{Float64,2}:
 33.0504  33.0604  33.0704  33.0804  …  28.18  28.19  28.2  28.2  28.21

In [209]:
d = Normal(0,0.01)
pdf(d,0.09)

1.0279773571668916e-16

In [163]:
# How about we try using 2 frames later so that the timestep is double
# Maybe even try 3 frames. The hope is that idm can be somewhere close by
# This needs to be interesting as this is automotive driving research

0.08*0.08*0.2

0.00128

In [164]:
0.04*0.04*0.2

0.00032

In [160]:
pdf(Normal(298.8,0.032),298.7)

0.09444556359975141

In [137]:
function update_p_one_step(roadway,f,trupos,p_set_dict;car_id=-1)
    if car_id==-1 @show "Provide valid car_id" end
    
    lkhd_vec,p_mat,params = compute_particle_likelihoods(roadway,
        f,trupos,p_set_dict,car_id=car_id)
    
    @show log.(lkhd_vec)
    
    num_params = size(p_mat)[1]
    num_p = size(p_mat)[2]
    
    p_weight_vec = weights(lkhd_vec./sum(lkhd_vec)) # Convert to weights form to use julia sampling
    idx = sample(1:num_p,p_weight_vec,num_p)
    new_p_mat = p_mat[:,idx] #Careful that idx is (size,1) and not (size,2)
        
    new_p_set_dict = to_dict_form(params,new_p_mat)
    return new_p_set_dict
end

update_p_one_step (generic function with 1 method)

In [138]:
scene = Scene(500)
get!(scene,traj_highd,3)
# We are focusing on vehicle id 3 hence the findfirst(3,scene)
trupos = scene.entities[findfirst(3,scene)].state.posG.x
@show trupos

p_set_dict = gen_test_particles(5)
@show p_set_dict
scene = Scene(500)
get!(scene,traj_highd,1)
@show scene.entities[findfirst(3,scene)].state.v
p_set_new = update_p_one_step(roadway_highd,scene,trupos,p_set_dict,car_id=6)

trupos = 298.77000956064006
p_set_dict = Dict(:v_des=>[33.0, 33.5, 33.0, 33.5, 33.5],:σ=>[17.5, 11.3, 14.7, 11.9, 17.2])
((scene.entities[findfirst(3, scene)]).state).v = 33.050438271060955
log.(lkhd_vec) = [-Inf, -Inf, -Inf, -Inf, -Inf]


Dict{Any,Any} with 2 entries:
  :v_des => [33.0, 33.0, 33.0, 33.0, 33.0]
  :σ     => [17.5, 17.5, 17.5, 17.5, 17.5]

# Filtering functions

In [122]:
function compute_particle_likelihoods(roadway,f,trupos,p_set_dict;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    timestep = 0.04 #TODO: Remove hardcoding, 0.04 is for highd
    p_mat, params, vec_val_vec = to_matrix_form(p_set_dict)
    
    num_params=size(p_mat)[1]
    num_p = size(p_mat)[2]
    lkhd_vec = Array{Float64}(undef,num_p)
    for i in 1:num_p    
        # Create dict version for a single particle
        p_dict = Dict()
        for j in 1:num_params
            p_dict[params[j]]=vec_val_vec[j][i]
        end
        
        std_dev_acc = p_dict[:σ]
        
        # hack to avoid the std_dev_pos become negative and error Normal distb
        if std_dev_acc <= 0 std_dev_acc = 0.1 end
        
        # TODO: This math needs to be verified from random variable calculations
        std_dev_pos = timestep*timestep*std_dev_acc
        
        hpos = hallucinate_a_step(roadway,f,p_dict,car_id=car_id)
        
        lkhd_vec[i] = pdf(Normal(hpos,std_dev_pos),trupos[1])
    end

    return lkhd_vec,p_mat,params
end

compute_particle_likelihoods (generic function with 1 method)

In [113]:
# Test for compute_particle_likelihoods
v_array = [10.,15.,20.,25.,30.]
num_p = length(v_array)
sig_array = [0.1,0.3,0.2,0.4,0.5]	
p_set_dict = Dict(:v_des=>v_array,:σ=>sig_array)
td1 = traj_highd;
scene = Scene(500)
get!(scene,td1,1)
roadway = roadway_highd;
    # trupos is a misnomer here as it is generated by hallucination
    # and is not the real data
trupos = hallucinate_a_step(roadway,scene,Dict(:v_des=>25.0,:σ=>0.0),
    car_id=3)
    
lkhd_vec,p_mat,params = compute_particle_likelihoods(roadway,scene,
    trupos,p_set_dict,car_id=3)
@show lkhd_vec
@test length(lkhd_vec) == num_p
@test length(params) == 2
@test size(p_mat)[1] == 2
@test size(p_mat)[2] == 5
@test any(isnan,lkhd_vec) == false #Gets triggered when wrong car_id called
# For example only 1 car on road but you say car_id = 2

lkhd_vec = [2313.56, 745.318, 1122.89, 533.086, 466.748]


Test Passed

# Driving functions

In [195]:
"""
# Example:
scene = Scene(500)
get!(scene,traj_highd,1)
vehid = scene.entities[2].id
@show vehid
vehicle = scene.entities[findfirst(scene.entities[2].id,scene)]
print("Location before hallucination = $(vehicle.state.posG.x)\n")
particle = Dict(:v_des=>25.0,:σ=>0.5)
hall_pos = hallucinate_a_step(roadway_highd,scene,
    particle,car_id=scene.entities[2].id)
print("Hallucinated position = $(hall_pos[1])\n")
scene = Scene(500)
get!(scene,traj_highd,2)
vehnew = scene.entities[findfirst(vehid,scene)]
print("True position in timestep 2 = $(vehnew.state.posG.x)")
"""
function hallucinate_a_step(roadway,scene_input,particle;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    
    scene = deepcopy(scene_input)
    #scene = scene_input # This was the failure case

    models = Dict{Int, DriverModel}()
    
    # Create driver models for all the cars in the scene
    for veh in scene
        if veh.id == car_id
            models[veh.id] = IntelligentDriverModel(;particle...)
        else
            # TODO: RESEARCH QUESTION: What drives the other vehicles
            models[veh.id] = IntelligentDriverModel(v_des=10.0)
        end
    end
    
    n_steps = 4
    dt = 0.04
    rec = SceneRecord(n_steps, dt)
    
    simulate!(rec, scene, roadway, models, n_steps)
    
    X = Array{Float64}(undef,n_steps, 1)

    for t in 1:n_steps
        f = rec.frames[n_steps - t + 1]
        
            # Access the vehicle with id as car_id and return its frenet s
        X[t,1] = f.entities[findfirst(car_id,f)].state.posG.x
    end
    @show X
    return X[1]
end

hallucinate_a_step

In [218]:
# Function to return the velocity after hallucination
# Idea is to combine this with position likelihood
function hallucinate_vel(roadway,scene_input,particle;car_id=-1)
    if car_id==-1 @show "Please give valid car_id" end
    
    scene = deepcopy(scene_input)
    #scene = scene_input # This was the failure case
    n_cars = scene.n 

    models = Dict{Int, DriverModel}()
    
    # Create driver models for all the cars in the scene
    for veh in scene
        if veh.id == car_id
            models[veh.id] = IntelligentDriverModel(;particle...)
        else
            # TODO: RESEARCH QUESTION: What drives the other vehicles in the hallucination
            models[veh.id] = IntelligentDriverModel(v_des=10.0)
        end
    end
    
    n_steps = 1
    dt = 0.1
    rec = SceneRecord(n_steps, dt)
    
    simulate!(rec, scene, roadway, models, n_steps)
    
    X = Array{Float64}(undef,n_steps, 1)

    for t in 1:n_steps
        f = rec.frames[n_steps - t + 1]
        
            # Access the vehicle with id as car_id and return its frenet s
        X[t,1] = scene.entities[findfirst(car_id,f)].state.v
    end
    return X[1]
end

hallucinate_vel (generic function with 1 method)

In [237]:
# Test for hallucinate_a_step
timestep = 0.04 # This is the highd timestep
test_num = 5 # The array element number in the scene. Gives vehicle id later
scene = Scene(500)
get!(scene,traj_highd,1)
vehid = scene.entities[test_num].id
@show vehid
vehicle = scene.entities[findfirst(scene.entities[test_num].id,scene)]
print("Location before hallucination = $(vehicle.state.posG.x)\n")
print("Velocity before hallucination = $(vehicle.state.v)\n")
sig_particle = 0.1
particle = Dict(:v_des=>35.0,:σ=>sig_particle)
hall_pos = hallucinate_a_step(roadway_highd,scene,particle,car_id=vehid)
hall_vel = hallucinate_vel(roadway_highd,scene,particle,car_id=vehid)
@show hall_pos,hall_vel
scene = Scene(500)
get!(scene,traj_highd,2)
vehicle = scene.entities[findfirst(vehid,scene)]
trupos = (vehicle.state.posG.x)
truvel = vehicle.state.v
ll_pos = log(pdf(Normal(hall_pos,timestep*timestep*sig_particle),trupos))
ll_vel = log(pdf(Normal(hall_vel,timestep*sig_particle),truvel))

# If log likelihood position is not infinity then we show the position
if isinf(ll_pos)
    @show ll_vel
else
    @show ll_pos + ll_vel
end

vehid = 10
Location before hallucination = 232.78000744896002
Velocity before hallucination = 32.39012607483481
X = [234.078; 235.379; 236.685; 237.994]
(hall_pos, hall_vel) = (234.07769046519323, 32.63345378813808)
ll_vel = -Inf


-Inf

In [228]:
?isinf

search: NonMatrixDistribution ContinuousMatrixDistribution

Couldn't find notisinf
Perhaps you meant nothing, isinf or notify


No documentation found.

Binding `notisinf` does not exist.


In [197]:
# See how 100% sampled results are only one particle
# This is particle deprivation in action
lkhd_vec = zeros(5,1)
@show lkhd_vec
num_p = 5
for i in 1:num_p
    lkhd_vec[i] = pdf(Normal(302.57,0.16*0.16*i*0.2),302.747)
end
@show lkhd_vec
p_weight_vec = weights(lkhd_vec./sum(lkhd_vec))
@show p_weight_vec
idx = sample(1:5,p_weight_vec,5)
@show make this happen

lkhd_vec = [0.0; 0.0; 0.0; 0.0; 0.0]
lkhd_vec = [2.38539e-258; 5.15335e-64; 3.79858e-28; 1.17476e-15; 6.48794e-10]
p_weight_vec = [3.67665e-249, 7.94296e-55, 5.85482e-19, 1.81068e-6, 0.999998]


5-element Array{Int64,1}:
 5
 5
 5
 5
 5

# Visualization functions

In [31]:
"""
# Example: make_video(range=1:20,filename="test.mp4")
"""
function make_video(;range=nothing,filename="output.mp4")

    frames = Frames(MIME("image/png"), fps=10)
    scene = Scene(500)
    for i in range
        temp_scene = get!(scene,traj_highd,i)

        overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, 
        pos=VecSE2(veh.state.posG.x+0.5,veh.state.posG.y+0.5)) for veh in scene]
        
        scene_visual = render(temp_scene, roadway_highd, 
            #cam=CarFollowCamera{Int}(2,5.0),
            #cam=StaticCamera(VecE2(1966400, 570900), 5.0),
            cam=FitToContentCamera(0.),
            overlays)
        push!(frames,scene_visual)
    end
    write(string("media/"*filename),frames)    
    
    return nothing
end

make_video (generic function with 1 method)

In [ ]:
# For overlaying hallucination on the ground truth
# Borrowed from visualization_functions.jl
struct my_overlay <: SceneOverlay
    scene::Scene
    color # Needs to be of form colorant"Colorname"
end

"""
Render method for `my_overlay`. Helpful for making the color choice
"""
function AutoViz.render!(rendermodel::RenderModel,overlay::my_overlay, 
        scene::Scene, roadway::Roadway)
    AutoViz.render!(rendermodel,overlay.scene,car_color = overlay.color)
    return rendermodel
end

# Code dump for use later/make functions

In [ ]:
# Drive using IDM
scene = Scene(500)
temp = get!(scene,traj_highd,1)
models = Dict{Int,DriverModel}()
for veh in scene
    models[veh.id] = IntelligentDriverModel(v_des=40.0)
end
n_steps = 100
dt = 0.04 # To match the timestep of traj_highd, ngsim was timestep 0.1
rec = SceneRecord(n_steps,dt)
simulate!(rec,scene,roadway_highd,models,n_steps)

In [ ]:
# highd replays with idm hallucination overlayed
frames = Frames(MIME("image/png"), fps=10)
scene = Scene(500)
for i in 1:100
    temp_scene = get!(scene,traj_highd,i)

    overlay_hallucination = my_overlay(rec[i-100],colorant"white")
    
    scene_visual = render(temp_scene, roadway_highd, 
        #cam=CarFollowCamera{Int}(2,5.0),
        #cam=StaticCamera(VecE2(1966400, 570900), 5.0),
        cam=FitToContentCamera(0.),
        [overlay_hallucination])
    push!(frames,scene_visual)
end
write("media/highd_halluc_vdes40.mp4",frames)